# Curso Análisis de Redes

## Cátedra de Teoría de las Telecomunicaciones - Universidad ORT Uruguay 


# Trabajo Obligatorio 2, curso 2020


------

### Datos de entrega:

* **Fecha de entrega:** 14/12/2020
* **Grupos:** Hasta 3 integrantes
* **Modalidad de entrega:** cuaderno en PDF en gestion + repositorio Github

-----

In [1]:
#using Pkg;Pkg.instantiate(); Pkg.update()

## Ejercicio 1

El objetivo de este ejercicio es verificar el modelo de Bianchi de WiFi en condiciones idealizadas (simuladas) y sobre una captura real de WiFi 802.11b realizada en el laboratorio.

### 1a. parte

El siguiente código permite simular en Julia el proceso de Backoff que vimos en WiFi. Se define un objeto (`mutable struct`) que almacena el estado de los contadores internos, y lleva la cuenta de la cantidad de intentos de acceso, éxitos y colisiones obtenidas.

Para construir una instancia, basta con realizar `station = BackoffProcess(CWmin,stages)` siendo `CWmin` la ventana mínima de backoff y `stages` el no. de duplicaciones. Por defecto todos los contadores se inicializan en $0$, la ventana en el mínimo y el contador se sortea uniforme.

In [3]:
using Distributions

mutable struct BackoffProcess
    
    CWmin::Integer ##minimal backoff window
    stages::Integer ##maximum number of duplications
    w::Integer ##current backoff counter
    s::Integer ##current stage
    attempts::Integer
    successes::Integer
    collisions::Integer
    
    BackoffProcess(CWmin::Integer,stages::Integer)= new(CWmin,stages,rand(DiscreteUniform(0,CWmin-1)),0,0,0,0)
end 

Los siguientes métodos permiten actualizar un objeto de tipo `BackoffProcess` en función de si lo que ocurre es un slot vacío, un slot de transmisión exitosa o una colisión.

In [4]:
function update_on_idle!(st::BackoffProcess)
    
    @assert st.w>0 "No deberia haber entrado aca ya que esta estación debía acceder"
    
    st.w=st.w-1
end

function update_on_success!(st::BackoffProcess)
    
    if st.w>0
        st.w=st.w-1
    else
        st.s=0
        st.w=rand(DiscreteUniform(0,st.CWmin-1))
        st.attempts=st.attempts+1
        st.successes=st.successes+1
    end
end

function update_on_collision!(st::BackoffProcess)
    
    if st.w>0
        st.w=st.w-1
    else
        st.s=min(st.s+1,st.stages)
        st.w=rand(DiscreteUniform(0,st.CWmin*2^st.s-1))
        st.attempts=st.attempts+1
        st.collisions=st.collisions+1
    end
end

update_on_collision! (generic function with 1 method)

Se pide:

1. Realice un código basado en los vistos en clase para simular una celda de $N$ estaciones intentando acceder utilizando el proceso de Backoff de WiFi con los parámetros habituales de 802.11b ($CW_{min}=32, CW_{max}=1024$) y los tiempos de slot, transmisión y colisión ya vistos en clase.


2. Simule lo anterior para el caso de $N=5$ estaciones transmitiendo a $1Mbps$ durante 60 segundos. Grafique la evolución temporal del sistema (por ejemplo paquetes transmitidos).


3. Estime de la simulación:
   * El throughput total de la celda (total de paquetes enviados exitosamente sobre tiempo transcurrido).
   * La probabilidad de acceso de cada estación.
   * La probabilidad condicional de colisión de cada estación.
   
   
4. Compare los resultados anteriores con los que surgen del modelo de Bianchi visto en clase.

### 2a. parte

En el archivo `datos/captura_wifi.csv` se dispone del resumen de una captura Wireshark de $N=5$ estaciones en un AP conectadas a $1Mbps$, intentando transmitir en condiciones de saturación. La captura se realizó del lado del Access Point. La misma puede leerse usando el comando `CSV.read("datos/captura_wifi.csv")` del paquete `CSV`. A su vez puede ser útil el paquete `DataFrames` para trabajar con el resultado. 

1. Filtre los datos correspondientes a las transmisiones de cada estación.
2. Grafique la cantidad de paquetes transmitidos en función del tiempo y compare entre estaciones.
3. Compare el throughput total obtenido con el resultante del modelo anterior y comente si el reparto fue equitativo.

## Ejercicio 2

El objetivo del siguiente ejercicio es analizar el algoritmo de control de congestión de TCP, en particular en su versión Reno y validar algunos de los modelos teóricos vistos en clase.

Para ello, se realizó una conexión directa entre dos máquinas Linux, y se utilizó la herramienta `tc` para convertir dicho enlace en un cuello de botella, limitando tanto la velocidad como el buffer (FIFO)
del mismo.

Se utilizó a su vez la herramienta `tcpprobe`, que permite extraer en tiempo real el valor de la ventana de congestión de TCP y del round trip time medido. Obteniéndose las trazas que se detallan más adelante.

El formato de la traza tcp probe es el siguiente:

`[time][src][dst][header length][next seq][last ack][cwnd][ssthresh][rcv wnd][srtt]`

El `srtt` se expresa en múltiplos de kernel ticks, que en el caso de Linux corresponde a 4ms.

Las trazas pueden leerse utilizando el comando `CSV.read(<archivo>, header=false)` (ya que no tienen encabezado). Los valores hexadecimales de no. de secuencia y ACK pueden convertirse a entero mediante el comando `parse(Int64,x)` siendo `x` un `String` con el valor.

### 1a. parte: Análisis de un flujo TCP

En esta parte, se analiza un único flujo TCP pasando por un cuello de botella de capacidad $C$, retardo de ida y vuelta (latencia) $d$ y buffer máximo $B$. Se proporcionan 3 trazas:    

   1. `traza1Ma.tr`: C = 1 Mbps, d = 200 ms, B = 6 paquetes.
   2. `traza1Mb.tr`: C = 1 Mbps, d = 200 ms, B = 17 paquetes.
   3. `traza10M.tr`: C = 10 Mbps, d = 200 ms, B = 20 paquetes.

Para los tres casos anteriores:

 1. Estime la ventana óptima y la ventana máxima que puede lograr una conexión en el enlace y compare con el comportamiento real logrado por TCP. 
 2. Grafique la evolución de los números de secuencia enviados y calcule el throughput logrado por la conexión a partir de los mismos. Verifique si TCP logra utilizar toda la capacidad del enlace.
 3. Para las trazas 1 y 2, verifique la fórmula de Mathis:
 
     $$x = \frac{L}{RTT}\sqrt{\frac{1.5}{\beta p}},$$

siendo $p$ la tasa de pérdida de paquetes. El parámetro $\beta$ es la cantidad de paquetes por ACK en el caso
en que se utiliza delayed-ACK. En Linux, $\beta=2$. Para estimar $p$, asuma que se pierde un paquete por cada
evento de congestión observado.


### 2a. parte: Justicia en TCP

En esta parte, se analizan varias conexiones TCP en paralelo. pasando por un cuello de botella de capacidad $C$, retardo de ida y vuelta (latencia) $d$ y buffer máximo $B$. Se proporciona una traza:

`traza5M.tr`: C = 5 Mbps, d = 200 ms, B = 40 paquetes.

Calcule la cantidad máxima de paquetes que admite esta configuración, y observe cómo se reparte la
ventana entre las conexiones. ¿Se logra un reparto justo? Verifique lo anterior calculando el throughput
de cada conexión a partir de los nos. de secuencia.